In [ ]:
import numpy as np
import pandas as pd

dist_matrix = pd.read_csv('Matriz_de_Distancias.csv', header=None)
dist_matrix

dist_matrix_clean = dist_matrix.drop([0], axis=0).drop([0], axis=1).reset_index(drop=True)
dist_matrix_clean = dist_matrix_clean.astype(float)
dist_matrix_clean
dist_matrix = dist_matrix_clean.values
N_CITIES = dist_matrix.shape[0]

# Parámetros del AG
POP_SIZE = 100
GENERATIONS = 500
TOURNAMENT_SIZE = 3
MUT_RATE = 0.2
MAX_NO_IMPROVE = 50
CODING = 'entero'  # 'entero', 'real', 'binario'
EXPERIMENTS = 10


def route_cost(route):
    total = 0
    for i in range(N_CITIES):
        total += dist_matrix[route[i], route[(i + 1) % N_CITIES]]  
    return total


def decode(chrom):
    if CODING == 'entero':
        return chrom
    elif CODING == 'real':
        return np.argsort(chrom)
    elif CODING == 'binario':
        num_bits = int(np.ceil(np.log2(N_CITIES)))
        values = [int(''.join(str(b) for b in chrom[i * num_bits:(i + 1) * num_bits]), 2) % N_CITIES for i in range(N_CITIES)]
        _, perm = np.unique(values, return_index=True)  
        route = np.array(values)[sorted(perm)]
        missing = [c for c in range(N_CITIES) if c not in route]
        return np.concatenate([route, missing])
        
# población inicial
def init_population():
    if CODING == 'entero':
        return [np.random.permutation(N_CITIES) for _ in range(POP_SIZE)]
    elif CODING == 'real':
        return [np.random.rand(N_CITIES) for _ in range(POP_SIZE)]
    elif CODING == 'binario':
        num_bits = int(np.ceil(np.log2(N_CITIES)))
        return [np.random.randint(0, 2, N_CITIES * num_bits) for _ in range(POP_SIZE)]

# torneo
def tournament_selection(pop, costs):
    selected = np.random.choice(len(pop), TOURNAMENT_SIZE, replace=False)
    winner = selected[np.argmin([costs[i] for i in selected])]
    return pop[winner]

# cruza
def crossover(p1, p2):
    size = len(p1)
    if CODING == 'entero':
        a, b = sorted(np.random.choice(range(size), 2, replace=False))
        child = -np.ones(size, dtype=int)
        child[a:b+1] = p1[a:b+1]
        fill = [city for city in p2 if city not in child]
        idx = 0
        for i in range(size):
            if child[i] == -1:
                child[i] = fill[idx]
                idx += 1
        return child
    else:
        cp = np.random.randint(1, size - 1)
        return np.concatenate([p1[:cp], p2[cp:]])

# swap mutación
def mutate(chromo):
    if np.random.rand() < MUT_RATE:
        if CODING == 'entero':
            i, j = np.random.choice(N_CITIES, 2, replace=False)
            chromo[i], chromo[j] = chromo[j], chromo[i]
        elif CODING == 'real':
            i, j = np.random.choice(N_CITIES, 2, replace=False)
            chromo[i], chromo[j] = chromo[j], chromo[i]
        elif CODING == 'binario':
            # Swap bits de dos genes
            num_bits = int(np.ceil(np.log2(N_CITIES)))
            i, j = np.random.choice(N_CITIES, 2, replace=False)
            start_i, start_j = i * num_bits, j * num_bits
            chromo[start_i:start_i + num_bits], chromo[start_j:start_j + num_bits] = chromo[start_j:start_j + num_bits], chromo[start_i:start_i + num_bits].copy()
    return chromo


for exp in range(1, EXPERIMENTS + 1):
    population = init_population()
    best_cost = float('inf')
    no_improve = 0

    for gen in range(GENERATIONS):
        
        decoded = [decode(c) for c in population]
        costs = [route_cost(r) for r in decoded]
        elite_idx = np.argmin(costs)
        elite = population[elite_idx]
        current_best = costs[elite_idx]

        if current_best < best_cost:
            best_cost = current_best
            no_improve = 0
        else:
            no_improve += 1

        next_gen = [elite.copy()]
        while len(next_gen) < POP_SIZE:
            p1 = tournament_selection(population, costs)
            p2 = tournament_selection(population, costs)
            child = crossover(p1, p2)
            child = mutate(child)
            next_gen.append(child)

        population = next_gen

    else:
        print(f"Exp {exp}: Máximo de generaciones alcanzado | Mejor costo: {best_cost}")


Exp 1: Máximo de generaciones alcanzado | Mejor costo: 2085.0
Exp 2: Máximo de generaciones alcanzado | Mejor costo: 2090.0
Exp 3: Máximo de generaciones alcanzado | Mejor costo: 2158.0
Exp 4: Máximo de generaciones alcanzado | Mejor costo: 2103.0
Exp 5: Máximo de generaciones alcanzado | Mejor costo: 2210.0
Exp 6: Máximo de generaciones alcanzado | Mejor costo: 2095.0
Exp 7: Máximo de generaciones alcanzado | Mejor costo: 2155.0
Exp 8: Máximo de generaciones alcanzado | Mejor costo: 2103.0
Exp 9: Máximo de generaciones alcanzado | Mejor costo: 2090.0
Exp 10: Máximo de generaciones alcanzado | Mejor costo: 2167.0
